In [30]:
import pandas as pd
import json

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))
team_id_list = [v for v in team_dic.keys()]


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り
# 各々の回数をカウント
tmp = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
colnames = ['rat_full_play', 'rat_out_play', 'rat_in_play', 'rat_inout_play', 'rat_bench_play', 'rat_susp_play']
characters = ['○', '▲', '▽', '◆', 'B', '×']


#記号を出場時間に変換する関数を定義
def convert_sign(self):
    import re
    
    if "○" in self:
        return int(90)
    
    elif "▽" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)

    elif "▲" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)    
    
    elif "◆" in self:        
            num = re.sub(r"\D", "", self)
            return int(num)
    
    else:
        return int(0)

pretime_2018 = pd.DataFrame({"name":[], "pretime":[]})
pretime_2017 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2016 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2015 = pd.DataFrame({"name":[], "pretime":[]})    

df_dic = {2018:pretime_2018, 2017:pretime_2017, 2016:pretime_2016, 2015:pretime_2015}
for year in [2018,2017,2016,2015]:
    # 2015シーズンのjsonをcsvに変換
    #year = 2015
    team_df_year = team_df[team_df.year == year]
    df = pd.DataFrame({"name":[],
                     "pretime":[]})
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')

        else:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

        #４シーズンに分けるための数値作成
        num_to_div_list = [(num_match//4)*n for n in range(1,5)]
        first = num_to_div_list[0]
        second = num_to_div_list[1]
        third = num_to_div_list[2]
        forth = num_match
        
        #4シーズンに分ける
        full_df = full_df.fillna("0")    
        full_df_con = full_df.applymap(convert_sign)
        full_df_con["pretime"] = full_df_con.sum(axis=1)
        full_df_con["1st_pretime"] = full_df_con.iloc[:, :first].sum(axis=1)
        full_df_con["2st_pretime"] = full_df_con.iloc[:, first:second].sum(axis=1)
        full_df_con["3st_pretime"] = full_df_con.iloc[:, second:third].sum(axis=1)
        full_df_con["4st_pretime"] = full_df_con.iloc[:, third:num_match].sum(axis=1)

        new_df = pd.DataFrame()
        new_df["name"] = full_df_con.index
        new_df["pretime"] = full_df_con["pretime"].values
        new_df["1st_pretime"] = full_df_con["1st_pretime"].values
        new_df["2st_pretime"] = full_df_con["2st_pretime"].values
        new_df["3st_pretime"] = full_df_con["3st_pretime"].values
        new_df["4st_pretime"] = full_df_con["4st_pretime"].values
        new_df['team_id'] = team_id
        new_df['team'] = new_df['team_id'].map(int).map(team_dic)
        df = pd.concat([df, new_df])

 
    df["name-team"] = df["name"] + "-" + df["team"]
    df.drop(["name"], axis=1, inplace=True)
    df.drop(["team"], axis=1, inplace=True)


    #出場時間を出場割合に変換
    for i in ["", "1st_", "2st_", "3st_", "4st_"]:
        df[i + "time_rate"] = df[i + "pretime"].copy()
        df[i + "time_rate_private"] = df[i + "pretime"].copy()

    for i in team_id_list:
        for j in ["", "1st_", "2st_", "3st_", "4st_"]:
                    s = df.loc[df["team_id"]==i, j + "pretime"].sum()
                    df.loc[df["team_id"]==i, j + "time_rate"] /= (s/100)
                    
    
    for j in ["", "1st_", "2st_", "3st_", "4st_"]:
        df[j + "time_rate_private"] /= (3060/100)
    
    for i in ["pretime", "1st_pretime", "2st_pretime", "3st_pretime", "4st_pretime", "team_id"]:
        df.drop([i], axis=1, inplace=True)

    df_dic[year] = df
    
pretime_2018 = df_dic[2018]
pretime_2017 = df_dic[2017]
pretime_2016 = df_dic[2016]
pretime_2015 = df_dic[2015]

In [32]:
pretime_2018.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2018改.csv", index=False)
pretime_2017.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2017改.csv", index=False)
pretime_2016.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2016改.csv", index=False)
pretime_2015.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2015改.csv", index=False)


In [9]:
a = pd.read_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2018.csv")
a.head(3)

,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,クォン スンテ-鹿島,7.222041,9.090909,6.829392,4.545455,8.181818
1,内田 篤人-鹿島,1.789164,2.032828,4.122929,1.123737,0.262626
2,昌子 源-鹿島,4.095462,6.818182,5.526748,0.517677,3.636364


In [31]:
pretime_2018

,name-team,time_rate,time_rate_private,1st_time_rate,1st_time_rate_private,2st_time_rate,2st_time_rate_private,3st_time_rate,3st_time_rate_private,4st_time_rate,4st_time_rate_private
0,クォン スンテ-鹿島,7.222041,79.411765,9.090909,23.529412,6.829392,17.647059,4.545455,11.764706,8.181818,26.470588
1,内田 篤人-鹿島,1.789164,19.673203,2.032828,5.261438,4.122929,10.653595,1.123737,2.908497,0.262626,0.849673
2,昌子 源-鹿島,4.095462,45.032680,6.818182,17.647059,5.526748,14.281046,0.517677,1.339869,3.636364,11.764706
3,レオ シルバ-鹿島,4.615568,50.751634,2.310606,5.980392,3.515872,9.084967,7.954545,20.588235,4.666667,15.098039
4,植田 直通-鹿島,3.744762,41.176471,9.090909,23.529412,6.829392,17.647059,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
30,永坂 勇人-水戸,0.024072,0.326797,0.000000,0.000000,0.101010,0.326797,0.000000,0.000000,0.000000,0.000000
31,伊藤 涼太郎-水戸,4.111502,55.816993,0.740140,2.385621,3.414141,11.045752,6.868687,22.222222,5.194040,20.163399
32,バティスタ-水戸,0.982139,13.333333,0.000000,0.000000,0.000000,0.000000,0.333333,1.078431,3.156831,12.254902
33,齋藤 恵太-水戸,0.524770,7.124183,0.892223,2.875817,1.101010,3.562092,0.141414,0.457516,0.058928,0.228758


In [13]:
new_df.head(10)

,name,pretime,1st_pretime,2st_pretime,3st_pretime,4st_pretime,team_id,team
0,本間 幸司,3105,630,495,900,1080,94,水戸
1,田向 泰輝,2559,236,357,899,1067,94,水戸
2,田中 雄大,3762,899,900,883,1080,94,水戸
3,山﨑 貴雅,1,1,0,0,0,94,水戸
4,金 聖基,1450,720,634,96,0,94,水戸
5,石神 幸征,1470,621,231,2,616,94,水戸
6,鈴木 雄斗,1789,384,291,88,1026,94,水戸
7,岩尾 憲,3690,900,900,900,990,94,水戸
8,小谷野 顕治,373,92,270,0,11,94,水戸
9,船谷 圭祐,2375,88,874,568,845,94,水戸


In [17]:
s

11880.0

In [18]:
df

,name-team,time_rate,time_rate_private,1st_time_rate,1st_time_rate_private,2st_time_rate,2st_time_rate_private,3st_time_rate,3st_time_rate_private,4st_time_rate,4st_time_rate_private
0,林 卓人-広島,9.090909,11.00000,9.090909,2.588235,9.090909,2.588235,9.090909,2.588235,9.090909,3.235294
1,水本 裕貴-広島,8.713607,11.00000,9.090909,2.588235,9.090909,2.588235,8.914141,2.588235,7.949495,3.235294
2,千葉 和彦-広島,9.090909,11.00000,9.090909,2.588235,9.090909,2.588235,9.090909,2.588235,9.090909,3.235294
3,青山 敏弘-広島,8.767083,11.00000,9.090909,2.588235,7.954545,2.588235,8.851010,2.588235,9.090909,3.235294
4,森﨑 浩司-広島,1.301248,11.00000,4.684343,2.588235,0.000000,2.588235,0.694444,2.588235,0.121212,3.235294
...,...,...,...,...,...,...,...,...,...,...,...
26,福留 健吾-水戸,0.000000,13.58366,0.000000,3.234967,0.000000,3.235294,0.000000,3.235294,0.000000,3.878105
27,石川 大徳-水戸,1.318385,13.58366,0.000000,3.234967,0.000000,3.235294,3.151515,3.235294,1.988708,3.878105
28,吉田 眞紀人-水戸,4.303998,13.58366,7.536115,3.234967,5.141414,3.235294,4.949495,3.235294,0.370776,3.878105
29,馬場 賢治-水戸,8.090747,13.58366,7.515911,3.234967,8.121212,3.235294,9.060606,3.235294,7.735738,3.878105
